# How do I create an _app_ from a CWL JSON?
## Overview
You can upload any app to the Platform using its raw [Common Workflow Language](http://www.commonwl.org/) description written in JSON. This recipe is actually an _edge-case_ but is one that the author uses frequently. It would also be a good fit if you are using code from a github repo, e.g. [Gaurav Kaushik's](https://github.com/gaurav-kaushik/py2cwl) excellent ones. 

## Objective
We'll **upload an app** to a project from its raw CWL. [[reference](http://docs.sevenbridges.com/docs/add-an-app-using-raw-cwl)]

## Procedure

 1. We'll start by creating a new project.
 2. Next, we'll check that there are no apps in our new project.
 3. Then, we'll upload our app using raw CWL.
 4. Lastly, we'll check that our new exists.

### Prerequisites
 1. You need your **authentication token** and the API needs to know about it. See <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a> for details. Learn more about [obtaining your authentication token](http://docs.sevenbridges.com/v1.0/docs/get-your-authentication-token).
 2. You have a **properly formatted** CWL description in a JSON file <sup>1</sup>.
 3. You understand how to list projects of which you are a member. We will use this call to check that the new project we will create does not already exist. [[recipe](projects_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-your-projects)]
 4. You undersand how to create a new project [[recipe](projects_MakeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)]
 4. You understand how to list apps in your project. We will first use this to check there are no apps in your project. Then, we will use this request again later to check you have successfully uploaded your new app to your project. [[recipe](apps_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-apps-available-to-you)]
 
Note that as with all okAPI recipes, this recipe makes use of the [Seven Bridges Public API Python library](../Tutorials/SBPLAT/Setup_API_environment.ipynb).

<sup>1</sup> **Note:** Due to slight inconsistencies in how the OS handles formatting, you might improperly copy the JSON if you manually **select**, **copy**, and **paste** code into a text editor before _saving_ it as a JSON. Instead, try to either download a properly formatted JSON or use the **Export** button on the desired app on the Seven Bridges Platform before saving the resulting webpage as a JSON directly. 

 
## Imports
We import the `Api` class from the official `sevenbridges-python` bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>.

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Create a new project
To avoid any copy-errors with the app, we will make a new project. First, we'll check if our new project's name is already taken. If this project name already exists, the code below will raise an interupt and fail. Be _creative_ with your project names, it's something you will look back on and laugh. Then, we create our new project

#### PROTIPS
This next cell is more extensively detailed in the following: [[recipe](projects_makeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)].

In [ ]:
# [USER INPUT] Set project name and billing group index here:
new_project_name = 'Shiny and new'                          
index_billing = -1   


# Check if this project already exists. LIST all projects and check for name match
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == new_project_name]      
              
if my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('A project named {} exists, please choose a unique name'
          .format(new_project_name))
    raise KeyboardInterrupt
else:
    # Create a new project
    # What are my funding sources?
    billing_groups = api.billing_groups.query()  
    print((billing_groups[index_billing].name + \
           ' will be charged for computation and storage (if applicable)'))

    # Set up the information for your new project
    new_project = {
            'billing_group': billing_groups[index_billing].id,
            'description': """A project created by the API recipe (apps_installFromJSON).
                          This also supports **markdown**
                          _Pretty cool_, right?
                       """,
            'name': new_project_name
    }

    my_project = api.projects.create(
        name = new_project['name'], billing_group = new_project['billing_group'], 
        description = new_project['description']
    )
    
    # (re)list all projects, and get your new project
    my_project = [p for p in api.projects.query(limit=100).all() 
              if p.name == new_project_name][0]

    print('Your new project {} has been created.'.format(
        my_project.name))
    # Print description if it exists
    if hasattr(my_project, 'description'): 
        print('Project description: \n {}'.format(my_project.description)) 

## Sanity-check: do I have any apps in my new project?
Since you have just created the project, there will be no **Files**, **Apps**, or **Tasks** in it. But just to be sure, let's query the apps in our project.

#### PROTIPS
This next cell is more extensively detailed in the following: [[recipe](apps_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-apps-available-to-you)].

In [ ]:
# List the apps in my project    
my_apps = api.apps.query(project = my_project.id, limit=100)
print('In project {}, you have {} apps.'.format(
    my_project.name, my_apps.total))

## Upload the JSON
We've pre-loaded a JSON for you in the **/app_jsons/** folder. We use the **json** library to manage the formatting. Here you should choose an *app\_id*. You can name this whatever you like, but please do _not_ use the same ID twice. If you do, your call will **fail** with _"Conflict: Requested app/revision already exists."_

Finally, we will list the apps in the project once more to check that it has sucessfully uploaded.

In [ ]:
# [USER INPUT] Specify the ID to assign to your app:
app_id = '/my-samtools-view'


#Load the Application JSONs
import json

f = open('app_jsons/samtools-view.json', 'r')
app_raw = f.read()
app = json.loads(app_raw)

# Create the Workflows
a_id = (my_project.id + app_id)
my_app_first = api.apps.install_app(id = a_id, raw = app)

# List the apps in my project    
my_apps = api.apps.query(project = my_project.id, limit=100)
print('In project {}, you have {} apps.'.format(
    my_project.name, my_apps.total))

for a in my_apps.all(): 
    print('App name is {}; \t App id: {}\n'.format(
        a.name, a.id))

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/add-an-app-using-raw-cwl).